# Availability Management
Format de fichier : Who, Date, Available

## Internals

In [ ]:
_path=''

In [16]:
import pandas as pd
import datetime as dt
import numpy as np

In [23]:
def InitAvailability(start,end, who):
    avail = np.zeros(shape=((end-start).days,len(who)))    
    dateRange = np.arange(start,end, dtype='datetime64[D]')
    return pd.DataFrame(data=avail, columns=who,index=dateRange)    

In [31]:
def SetAvailable(avail, weekday, who, amount):
    None
def SetOnHoliday(avail, dateFrom,dateTo,who):
    avail.loc[dateFrom:dateTo,who]=0

In [25]:
def SaveToCsv(path, avail):
    avail.to_csv(path_or_buf=path)

## Actuals

In [27]:
_avail = InitAvailability(dt.datetime(2016,11,21), dt.datetime(2017,06,30),['ACH','CGA','PTI','QPI'])

In [34]:
_avail.loc[:,'CGA']=1
_avail.head(10)

,ACH,CGA,PTI,QPI
2016-11-21 00:00:00,0.0,1,0.0,0.0
2016-11-22 00:00:00,0.0,1,0.0,0.0
2016-11-23 00:00:00,0.0,1,0.0,0.0
2016-11-24 00:00:00,0.0,1,0.0,0.0
2016-11-25 00:00:00,0.0,1,0.0,0.0
2016-11-26 00:00:00,0.0,1,0.0,0.0
2016-11-27 00:00:00,0.0,1,0.0,0.0
2016-11-28 00:00:00,0.0,1,0.0,0.0
2016-11-29 00:00:00,0.0,1,0.0,0.0
2016-11-30 00:00:00,0.0,1,0.0,0.0


In [35]:
SetOnHoliday(_avail,dt.datetime(2016,11,23),dt.datetime(2016,11,27),'CGA')
_avail.head(15)

,ACH,CGA,PTI,QPI
2016-11-21 00:00:00,0.0,1,0.0,0.0
2016-11-22 00:00:00,0.0,1,0.0,0.0
2016-11-23 00:00:00,0.0,0,0.0,0.0
2016-11-24 00:00:00,0.0,0,0.0,0.0
2016-11-25 00:00:00,0.0,0,0.0,0.0
2016-11-26 00:00:00,0.0,0,0.0,0.0
2016-11-27 00:00:00,0.0,0,0.0,0.0
2016-11-28 00:00:00,0.0,1,0.0,0.0
2016-11-29 00:00:00,0.0,1,0.0,0.0
2016-11-30 00:00:00,0.0,1,0.0,0.0


In [24]:
help (pd.DataFrame.to_csv)

Help on method to_csv in module pandas.core.frame:

to_csv(self, path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.', **kwds) unbound pandas.core.frame.DataFrame method
    Write DataFrame to a comma-separated values (csv) file
    
    Parameters
    ----------
    path_or_buf : string or file handle, default None
        File path or object, if None is provided the result is returned as
        a string.
    sep : character, default ','
        Field delimiter for the output file.
    na_rep : string, default ''
        Missing data representation
    float_format : string, default None
        Format string for floating point numbers
    columns : sequence, optional
        Columns to write
    header : boolean or list of string,